In [1]:

################### RUN THIS CELL


# Load the Drive helper and mount
# from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
# drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = './Dataset'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107



train_videos=[]
train_labels=[]
import traceback
import random

num = 1

for i,folder in enumerate(labels):
    try:
      for video in os.listdir(data_path+'/'+folder):
        print(num)
        num+=1
        video = os.path.join(data_path+'/'+folder+'/'+video)
        # Open the video file
        cap = cv2.VideoCapture(video)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select a random starting frame number
        start_frame = random.randint(0, total_frames - 16)

        # Set the number of frames to be selected
        num_frames = 16

        # Set the frame number to start with
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Loop through the frames and save them
        frames = []
        for j in range(num_frames):
            ret, image = cap.read()
            if ret:
              frames.append(cv2.resize(image,(112,112)))

        # Release the video file
        cap.release()

        train_videos.append(frames)
        train_labels.append(i)

    except Exception:
      traceback.print_exc()
train_videos = np.asarray(train_videos)
train_labels = np.asarray(train_labels).astype('int64')

# import numpy as np
# arr_reshaped = train_videos.reshape(train_videos.shape[0], -1) #1151
# np.savetxt("dataset.txt", arr_reshaped)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
# drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")



from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_videos,train_labels,test_size=0.25,shuffle=True)

input_shape =(32,16,112,112,3)
output_shape = 20

tf.keras.regularizers.L1L2(
    l1=0.01, l2=0.1
)

RGBmodel = Sequential()

# model.add(RandomFlip("horizontal"))
# model.add(RandomRotation(0.1))
# RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
RGBmodel.add(Flatten())

RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))

from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 400, validation_split = 0.2)


from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)

# Print the summary of the model
RGBmodel.summary()

Epoch 1/400
23/23 [==============================] - 69s 2s/step - loss: 2310.1086 - accuracy: 0.0468 - val_loss: 2289.6003 - val_accuracy: 0.0440
Epoch 2/400
23/23 [==============================] - 13s 589ms/step - loss: 2271.5596 - accuracy: 0.0303 - val_loss: 2251.4451 - val_accuracy: 0.0604
Epoch 3/400
23/23 [==============================] - 14s 593ms/step - loss: 2233.5471 - accuracy: 0.0371 - val_loss: 2213.6406 - val_accuracy: 0.0549
Epoch 4/400
23/23 [==============================] - 14s 594ms/step - loss: 2195.8853 - accuracy: 0.0619 - val_loss: 2176.1553 - val_accuracy: 0.0495
Epoch 5/400
23/23 [==============================] - 14s 595ms/step - loss: 2158.5481 - accuracy: 0.0523 - val_loss: 2138.9934 - val_accuracy: 0.0549
Epoch 6/400
23/23 [==============================] - 14s 594ms/step - loss: 2121.5491 - accuracy: 0.0536 - val_loss: 2102.1633 - val_accuracy: 0.0220
Epoch 7/400
23/23 [==============================] - 14s 594ms/step - loss: 2084.8687 - accuracy: 0.057

23/23 [==============================] - 14s 601ms/step - loss: 65.2770 - accuracy: 0.2875 - val_loss: 64.2596 - val_accuracy: 0.2308
Epoch 111/400
23/23 [==============================] - 14s 600ms/step - loss: 62.8450 - accuracy: 0.2999 - val_loss: 61.9063 - val_accuracy: 0.2088
Epoch 112/400
23/23 [==============================] - 14s 600ms/step - loss: 60.6165 - accuracy: 0.3356 - val_loss: 59.6795 - val_accuracy: 0.2308
Epoch 113/400
23/23 [==============================] - 14s 601ms/step - loss: 58.9208 - accuracy: 0.2765 - val_loss: 58.4189 - val_accuracy: 0.2198
Epoch 114/400
23/23 [==============================] - 14s 601ms/step - loss: 57.4299 - accuracy: 0.2847 - val_loss: 56.8192 - val_accuracy: 0.2582
Epoch 115/400
23/23 [==============================] - 14s 602ms/step - loss: 56.2322 - accuracy: 0.2820 - val_loss: 55.8466 - val_accuracy: 0.2143
Epoch 116/400
23/23 [==============================] - 14s 601ms/step - loss: 55.3680 - accuracy: 0.2806 - val_loss: 55.6208 -

23/23 [==============================] - 14s 601ms/step - loss: 48.0847 - accuracy: 0.5227 - val_loss: 48.3335 - val_accuracy: 0.3516
Epoch 221/400
23/23 [==============================] - 14s 601ms/step - loss: 48.0742 - accuracy: 0.4883 - val_loss: 48.2592 - val_accuracy: 0.3626
Epoch 222/400
23/23 [==============================] - 14s 601ms/step - loss: 48.0356 - accuracy: 0.4842 - val_loss: 48.3617 - val_accuracy: 0.4011
Epoch 223/400
23/23 [==============================] - 14s 601ms/step - loss: 47.9416 - accuracy: 0.5131 - val_loss: 48.3829 - val_accuracy: 0.3352
Epoch 224/400
23/23 [==============================] - 14s 601ms/step - loss: 47.9925 - accuracy: 0.4883 - val_loss: 48.0558 - val_accuracy: 0.4451
Epoch 225/400
23/23 [==============================] - 14s 601ms/step - loss: 47.8427 - accuracy: 0.5172 - val_loss: 48.3382 - val_accuracy: 0.4011
Epoch 226/400
23/23 [==============================] - 14s 601ms/step - loss: 47.8062 - accuracy: 0.5378 - val_loss: 48.1414 -

23/23 [==============================] - 14s 601ms/step - loss: 43.1800 - accuracy: 0.7001 - val_loss: 43.7071 - val_accuracy: 0.4615
Epoch 331/400
23/23 [==============================] - 14s 601ms/step - loss: 43.3377 - accuracy: 0.6602 - val_loss: 43.8474 - val_accuracy: 0.5000
Epoch 332/400
23/23 [==============================] - 14s 601ms/step - loss: 43.1550 - accuracy: 0.6836 - val_loss: 43.9830 - val_accuracy: 0.4286
Epoch 333/400
23/23 [==============================] - 14s 602ms/step - loss: 43.1631 - accuracy: 0.6754 - val_loss: 43.9740 - val_accuracy: 0.4341
Epoch 334/400
23/23 [==============================] - 14s 601ms/step - loss: 43.1153 - accuracy: 0.6905 - val_loss: 44.1022 - val_accuracy: 0.4341
Epoch 335/400
23/23 [==============================] - 14s 601ms/step - loss: 43.0022 - accuracy: 0.7139 - val_loss: 44.0768 - val_accuracy: 0.4725
Epoch 336/400
23/23 [==============================] - 14s 601ms/step - loss: 43.1185 - accuracy: 0.6795 - val_loss: 43.8211 -

Epoch 41/100
23/23 [==============================] - 17s 756ms/step - loss: 38.3593 - accuracy: 0.8845 - val_loss: 39.6018 - val_accuracy: 0.4945
Epoch 42/100
23/23 [==============================] - 14s 611ms/step - loss: 38.3215 - accuracy: 0.8955 - val_loss: 39.5949 - val_accuracy: 0.5055
Epoch 43/100
23/23 [==============================] - 17s 760ms/step - loss: 38.3396 - accuracy: 0.8721 - val_loss: 39.5975 - val_accuracy: 0.5055
Epoch 44/100
23/23 [==============================] - 14s 607ms/step - loss: 38.3430 - accuracy: 0.8721 - val_loss: 39.6305 - val_accuracy: 0.4780
Epoch 45/100
23/23 [==============================] - 14s 611ms/step - loss: 38.3191 - accuracy: 0.8803 - val_loss: 39.5934 - val_accuracy: 0.4945
Epoch 46/100
23/23 [==============================] - 15s 671ms/step - loss: 38.3328 - accuracy: 0.8693 - val_loss: 39.5538 - val_accuracy: 0.5165
Epoch 47/100
23/23 [==============================] - 14s 617ms/step - loss: 38.2923 - accuracy: 0.8776 - val_loss: 39

Epoch 97/100
23/23 [==============================] - 17s 732ms/step - loss: 38.1012 - accuracy: 0.8900 - val_loss: 39.4666 - val_accuracy: 0.5055
Epoch 98/100
23/23 [==============================] - 15s 657ms/step - loss: 38.0672 - accuracy: 0.9078 - val_loss: 39.4517 - val_accuracy: 0.5055
Epoch 99/100
23/23 [==============================] - 15s 660ms/step - loss: 38.0685 - accuracy: 0.9161 - val_loss: 39.4596 - val_accuracy: 0.5000
Epoch 100/100
23/23 [==============================] - 14s 628ms/step - loss: 38.1046 - accuracy: 0.8982 - val_loss: 39.4827 - val_accuracy: 0.5165
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 16, 56, 56, 64)    0         
_____________________________________________________________

In [3]:
RGBmodel.save('RGBmodel2.h5') 

In [ ]:
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)